# Import Packages

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

# We first load and merge the news data

In [6]:
tech_news = pd.read_csv('tech_news_processed.csv')
tech_news['date'] = pd.to_datetime(tech_news['date'])
tech_news

,Unnamed: 0,headline,category,short_description,date,Subjectivity,Polarity,Content Subjectivity,Content Polarity
0,0,Twitch Bans Gambling Sites After Streamer Scam...,TECH,One man's claims that he scammed people on the...,2022-09-21,0.000000,0.000000,0.450000,0.300000
1,1,TikTok Search Results Riddled With Misinformat...,TECH,A U.S. firm that monitors false online claims ...,2022-09-14,0.000000,0.000000,0.750000,-0.025000
2,2,Citing Imminent Danger Cloudflare Drops Hate S...,TECH,Cloudflare CEO Matthew Prince had previously r...,2022-09-05,0.900000,-0.800000,0.166667,-0.166667
3,3,Instagram And Facebook Remove Posts Offering A...,TECH,Facebook and Instagram began removing some of ...,2022-06-28,0.000000,0.000000,0.000000,0.000000
4,4,Google Engineer On Leave After He Claims AI Pr...,TECH,Artificially intelligent chatbot generator LaM...,2022-06-12,0.000000,0.000000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...
2098,2099,Google+ Now Open for Teens With Some Safeguards,TECH,"For the most part, teens' experience on Google...",2012-01-28,0.500000,0.000000,0.517857,0.354911
2099,2100,Web Wars,TECH,"These ""Web Wars"" threaten to rage on for some ...",2012-01-28,0.000000,0.000000,0.295833,0.041667
2100,2101,"First White House Chief Technology Officer, An...",TECH,He was appointed with a fair bit of fanfare as...,2012-01-28,0.207407,0.031481,0.411111,0.316667
2101,2102,Watch The Top 9 YouTube Videos Of The Week,TECH,If you're looking to see the most popular YouT...,2012-01-28,0.500000,0.500000,0.633333,0.366667


In [7]:
US_news = pd.read_csv('US_news_processed.csv')
US_news['date'] = pd.to_datetime(US_news['Date'])
US_news.drop(columns=['Unnamed: 0', 'Date'], inplace=True)
US_news

,Heading,Article,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...,0.000000,0.000000,0.377273,0.118182,2022-09-23
1,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...,0.200000,0.100000,0.000000,0.000000,2022-09-23
2,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...,0.266667,-0.233333,0.550000,-0.233333,2022-09-22
3,Cleaner Was Dead In Belk Bathroom For 4 Days B...,The 63-year-old woman was seen working at the ...,0.400000,-0.200000,0.225000,-0.200000,2022-09-22
4,Reporter Gets Adorable Surprise From Her Boyfr...,"""Who's that behind you?"" an anchor for New Yor...",0.750000,0.318182,0.577273,-0.131818,2022-09-22
...,...,...,...,...,...,...,...
1607,World Bank announces 57 bln in financing for Afri,strong>WASHINGTON: The World Bank on Sunday an...,0.000000,0.000000,0.416456,0.129798,2017-03-20
1608,Uber grounds self driving cars after accid,strong>WASHINGTON: Uber has grounded its fleet...,0.000000,0.000000,0.392063,-0.083730,2017-03-26
1609,Trump son in law 4 bn deal Chinese company,strong>WASHINGTON: A company owned by the fami...,0.000000,0.000000,0.388505,0.141968,2017-03-14
1610,World Bank announces 57 bln in financing for Afri,strong>WASHINGTON: The World Bank on Sunday an...,0.000000,0.000000,0.416456,0.129798,2017-03-20


In [8]:
business_news = pd.read_csv('business_news_processed.csv')
business_news['date'] = pd.to_datetime(business_news['date'])
business_news

,Unnamed: 0,headline,category,short_description,date,Subjectivity,Polarity,Content Subjectivity,Content Polarity
0,0,REI Workers At Berkeley Store Vote To Unionize...,BUSINESS,They follow in the footsteps of REI workers in...,2022-08-25,0.400000,0.800000,0.477273,0.068182
1,1,Twitter Lawyer Calls Elon Musk 'Committed Enem...,BUSINESS,Delaware Chancery Judge Kathaleen McCormick de...,2022-07-20,0.000000,0.000000,0.000000,0.000000
2,2,"Starbucks Leaving Russian Market, Shutting 130...",BUSINESS,Starbucks' move follows McDonald's exit from t...,2022-05-23,0.000000,0.000000,0.033333,0.000000
3,3,Crypto Crash Leaves Trading Platform Coinbase ...,BUSINESS,Cryptocurrency trading platform Coinbase has l...,2022-05-12,0.000000,0.000000,0.208333,-0.208333
4,4,"US Added 428,000 Jobs In April Despite Surging...",BUSINESS,"At 3.6%, unemployment nearly reached the lowes...",2022-05-06,0.000000,0.000000,0.283333,-0.033333
...,...,...,...,...,...,...,...,...,...
6882,7743,Four More Bank Closures Mark the Week of Janua...,BUSINESS,The general pattern of the FDIC closing banks ...,2012-01-28,0.500000,0.500000,0.562500,-0.162500
6883,7744,Everything You Need To Know About Overdraft Fe...,BUSINESS,Don't like keeping all of your money stuffed u...,2012-01-28,0.400000,0.000000,1.000000,0.250000
6884,7745,Walmart Waving Goodbye To Some Greeters,BUSINESS,"After 30 years, ""People Greeters"" will no long...",2012-01-28,0.000000,0.000000,0.500000,0.550000
6885,7746,"At World Economic Forum, Fear of Global Contag...",BUSINESS,"For decades, as crises have assailed developin...",2012-01-28,0.100000,0.100000,0.000000,0.000000


In [9]:
World_news = pd.read_csv("world_news_processed.csv")
World_news['date'] = pd.to_datetime(World_news['Date'])
World_news.drop(columns = ['Date'], inplace=True)
World_news

,Unnamed: 0,Heading,Article,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,0,Puerto Ricans Desperate For Water After Hurric...,More than half a million people remained witho...,1.000000,-0.600000,0.333333,0.166667,2022-09-22
1,1,Biden At UN To Call Russian War An Affront To ...,White House officials say the crux of the pres...,0.000000,0.000000,0.500000,-0.437500,2022-09-21
2,2,World Cup Captains Want To Wear Rainbow Armban...,FIFA has come under pressure from several Euro...,0.000000,0.000000,0.033333,0.000000,2022-09-21
3,3,Man Sets Himself On Fire In Apparent Protest O...,The incident underscores a growing wave of pro...,0.350000,0.050000,0.250000,0.250000,2022-09-21
4,4,Fiona Threatens To Become Category 4 Storm Hea...,Hurricane Fiona lashed the Turks and Caicos Is...,0.000000,0.000000,0.125000,-0.125000,2022-09-21
...,...,...,...,...,...,...,...,...
4341,4343,Chinese train manufacturer wins railcar bid in US,strong>BEIJING: A Chinese rail company has won...,0.100000,0.150000,0.398691,0.158471,2017-03-25
4342,4344,Laptop ban hits Dubai for 11m weekend traveller,strong>DUBAI: Dubai International Airport and ...,0.000000,0.000000,0.302778,0.043182,2017-03-25
4343,4345,Pak China relations not against any third coun...,"strong>BEIJING: Former Prime Minister, Shaukat...",0.000000,0.000000,0.173333,0.110000,2017-03-26
4344,4346,New Development Bank plans joint investments i...,strong>BEIJING: The New Development Bank plans...,0.454545,0.136364,0.408018,0.091746,2017-03-27


In [10]:
c1 = pd.read_csv("commodities1_processed.csv")
c1['date'] = pd.to_datetime(c1['date'])
c1.drop(columns = ['Unnamed: 0', 'year', 'month'], inplace=True)
c1

,title,publication,date,content,Subjectivity,Polarity,Content Subjectivity,Content Polarity
0,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,2017-01-06,"When Walt Disney’s “Bambi” opened in 1942, cri...",0.454545,0.136364,0.436291,0.067972
1,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,2017-04-10,"Death may be the great equalizer, but it isn’t...",0.477273,-0.031818,0.435646,0.129795
2,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,2016-12-31,"Just how is Hillary Kerr, the founder of ...",0.393939,0.193182,0.392158,0.089033
3,Weak Federal Powers Could Limit Trump’s Climat...,New York Times,2017-01-03,With Donald J. Trump about to take control of ...,0.539773,-0.119318,0.356313,0.068348
4,Can Carbon Capture Technology Prosper Under Tr...,New York Times,2017-01-05,"THOMPSONS, Tex. — Can one of the most promi...",0.454545,0.136364,0.422923,0.099423
...,...,...,...,...,...,...,...,...
7743,USA swim enemy’s risky alibi: Michael Phelps s...,New York Post,2016-08-12,Did Yulia Efimova really go there? The Russi...,0.000000,0.000000,0.436760,0.181104
7744,"Icahn Enterprises on the edge of junk, S&P warns",New York Post,2016-02-19,"Billionaire Carl Icahn, who last year may ne...",0.000000,0.000000,0.318452,-0.076786
7745,Socialite wants divorce moved to NYC in bid to...,New York Post,2016-07-30,A wealthy socialite wants a Manhattan judge to...,0.100000,0.200000,0.352189,0.131397
7746,Cops douse pipeline protesters in subfreezing ...,New York Post,2016-11-21,"CANNON BALL, N. D. — Authorities on Monday ...",0.000000,0.000000,0.372708,-0.112708


In [11]:
c2 = pd.read_csv("commodities2_processed.csv")
c2['date'] = c2['date'].astype('string')
c2['date'] = c2['date'].apply(lambda x: x.replace('/', '-'))
c2['date'] = pd.to_datetime(c2['date'])
c2.drop(columns = ['Unnamed: 0', 'year', 'month'], inplace=True)
c2

,title,publication,date,content,Subjectivity,Polarity,Content Subjectivity,Content Polarity
0,Santa: My paychecks sleigh,New York Post,2016-12-12,Being Santa isn’t always the jolliest or most ...,0.000000,0.000000,0.422540,0.168706
1,"SpaceX launches satellite, spectacularly botch...",New York Post,2016-01-18,"VANDENBERG AIR FORCE BASE, Calif. — A Space...",0.900000,0.600000,0.465238,0.053810
2,Broke ISIS fighters turn to peddling eggs on t...,New York Post,2016-05-03,ISIS jihadists have hatched a plan to raise ...,0.000000,0.000000,0.339556,-0.006444
3,Dr. Oz sued over claims about ‘fake’ imported ...,New York Post,2016-11-29,A trade group representing olive oil importers...,1.000000,-0.500000,0.363991,-0.003641
4,Qatar buys a piece of Empire State Building,New York Post,2016-08-24,Qatar’s sovereign wealth fund has made an icon...,0.000000,0.000000,0.354446,0.035128
...,...,...,...,...,...,...,...,...
7836,Our most memorable photos of 2016,Washington Post,2016-12-27,“The photo of people entrenched at Sta...,0.750000,0.500000,0.560095,0.152341
7837,"With new monuments in Nevada, Utah, Obama adds...",Washington Post,2016-12-28,President Obama on Wednesday created new...,0.454545,0.136364,0.403359,0.121572
7838,David Fahrenthold tells the behind-the-scenes ...,Washington Post,2016-12-29,"“Arnold and Tim, if you’d come up, we’re...",0.000000,0.000000,0.434710,0.065793
7839,2016 in Washington Post editorials,Washington Post,2016-12-29,"2016 was a bumpy year, and The Post’s editoria...",0.000000,0.000000,0.409185,0.036968


In [16]:
combined_news_1 = pd.concat([US_news, business_news, tech_news, World_news, c1, c2], axis=0)
combined_news_1.drop(columns=['Unnamed: 0', 'Heading', 'Article', 'short_description', 'headline', 'category', 'title', 'publication', 'content'], inplace = True)
combined_news_1

,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,0.000000,0.000000,0.377273,0.118182,2022-09-23
1,0.200000,0.100000,0.000000,0.000000,2022-09-23
2,0.266667,-0.233333,0.550000,-0.233333,2022-09-22
3,0.400000,-0.200000,0.225000,-0.200000,2022-09-22
4,0.750000,0.318182,0.577273,-0.131818,2022-09-22
...,...,...,...,...,...
7836,0.750000,0.500000,0.560095,0.152341,2016-12-27
7837,0.454545,0.136364,0.403359,0.121572,2016-12-28
7838,0.000000,0.000000,0.434710,0.065793,2016-12-29
7839,0.000000,0.000000,0.409185,0.036968,2016-12-29


In [68]:
combined_news_1['date'] = pd.to_datetime(combined_news_1['date'])

# oldest and newest date of all news sources
oldest_date = combined_news_1['date'].min()
most_recent_date = combined_news_1['date'].max()

datetime.datetime(2020, 3, 1, 0, 0)

In [70]:
# COVID times
covid_oldest_date = pd.to_datetime('2020-03-01')
covid_newest_date = pd.to_datetime('2022-12-31')

# Load the Options and Implied Volatility Data

In [22]:
spyIV = pd.read_csv('HistoricalIV_SPY.csv')

In [31]:
spyIV['Date'] = pd.to_datetime(spyIV['Date'])
spyIV = spyIV[(spyIV['Date'] >= oldest_date) & (spyIV['Date'] <= most_recent_date)].reset_index(drop=True)
spyIV

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,27.2500,23.3130,24.7004,21.9399,13803193.0
1,2022-09-22,25.2500,22.9190,24.6545,21.9436,8510835.0
2,2022-09-21,26.1400,22.6710,23.2973,21.8277,9723268.0
3,2022-09-20,25.1700,22.4105,23.6350,21.8820,6950163.0
4,2022-09-19,23.6600,22.1925,23.0622,21.8237,7426550.0
...,...,...,...,...,...,...
3369,2009-04-13,36.2173,38.5486,NaN,NaN,NaN
3370,2009-04-09,34.2882,38.9909,NaN,NaN,NaN
3371,2009-04-08,35.9798,39.4963,NaN,NaN,NaN
3372,2009-04-07,37.4447,40.0463,NaN,NaN,NaN


In [71]:
spyIV_COVID = spyIV[(spyIV['Date'] >= covid_oldest_date) & (spyIV['Date'] <= covid_newest_date)].reset_index(drop=True)
spyIV_COVID

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,27.250,23.3130,24.7004,21.9399,13803193.0
1,2022-09-22,25.250,22.9190,24.6545,21.9436,8510835.0
2,2022-09-21,26.140,22.6710,23.2973,21.8277,9723268.0
3,2022-09-20,25.170,22.4105,23.6350,21.8820,6950163.0
4,2022-09-19,23.660,22.1925,23.0622,21.8237,7426550.0
...,...,...,...,...,...,...
643,2020-03-06,35.768,20.6823,22.5562,13.3505,7057538.0
644,2020-03-05,33.466,19.5214,21.1487,13.1126,5020112.0
645,2020-03-04,25.034,18.4997,19.9014,12.8533,5396965.0
646,2020-03-03,32.011,17.9503,18.4344,12.6761,5445981.0


In [32]:
QQQ = pd.read_csv("HistoricalIV_QQQ.csv")

In [33]:
QQQ['Date'] = pd.to_datetime(QQQ['Date'])
QQQ = QQQ[(QQQ['Date'] >= oldest_date) & (QQQ['Date'] <= most_recent_date)].reset_index(drop=True)
QQQ

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,33.3500,30.3280,29.7307,29.4133,3819144.0
1,2022-09-22,31.5100,29.9660,29.6688,29.4100,2848974.0
2,2022-09-21,32.2900,29.7390,28.8371,29.3421,3428578.0
3,2022-09-20,31.6400,29.4970,29.3382,29.3909,1937901.0
4,2022-09-19,30.1700,29.2825,29.0248,29.3496,2561835.0
...,...,...,...,...,...,...
2876,2011-03-29,19.0443,19.3225,NaN,NaN,NaN
2877,2011-03-28,20.2398,19.3920,NaN,NaN,NaN
2878,2011-03-25,18.7910,19.1095,NaN,NaN,NaN
2879,2011-03-24,18.4833,19.2687,NaN,NaN,NaN


In [73]:
QQQ_COVID = QQQ[(QQQ['Date'] >= covid_oldest_date) & (QQQ['Date'] <= covid_newest_date)].reset_index(drop=True)
QQQ_COVID

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,33.350,30.3280,29.7307,29.4133,3819144.0
1,2022-09-22,31.510,29.9660,29.6688,29.4100,2848974.0
2,2022-09-21,32.290,29.7390,28.8371,29.3421,3428578.0
3,2022-09-20,31.640,29.4970,29.3382,29.3909,1937901.0
4,2022-09-19,30.170,29.2825,29.0248,29.3496,2561835.0
...,...,...,...,...,...,...
643,2020-03-06,39.998,24.8018,28.1611,17.2803,1169763.0
644,2020-03-05,37.492,23.6210,26.8479,17.0413,930637.0
645,2020-03-04,28.434,22.5849,25.6676,16.7968,1120002.0
646,2020-03-03,35.336,21.9879,24.0715,16.6002,1111158.0


In [34]:
GSG = pd.read_csv("HistoricalIV_GSG.csv")

In [35]:
GSG['Date'] = pd.to_datetime(GSG['Date'])
GSG = GSG[(GSG['Date'] >= oldest_date) & (GSG['Date'] <= most_recent_date)].reset_index(drop=True)
GSG

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,30.8900,30.6530,26.3471,29.2734,263.0
1,2022-09-22,28.6700,30.6240,25.7979,29.2481,26.0
2,2022-09-21,29.9600,30.6575,25.5120,29.2175,257.0
3,2022-09-20,29.7500,30.6805,25.9390,29.2308,96.0
4,2022-09-19,29.0400,30.7775,24.8089,29.1271,158.0
...,...,...,...,...,...,...
3370,2009-04-13,42.1140,43.2152,NaN,NaN,NaN
3371,2009-04-09,40.8940,43.3351,NaN,NaN,NaN
3372,2009-04-08,42.8479,43.4892,NaN,NaN,NaN
3373,2009-04-07,43.5973,43.6493,NaN,NaN,NaN


In [74]:
GSG_COVID = GSG[(GSG['Date'] >= covid_oldest_date) & (GSG['Date'] <= covid_newest_date)].reset_index(drop=True)
GSG_COVID

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,30.890,30.6530,26.3471,29.2734,263.0
1,2022-09-22,28.670,30.6240,25.7979,29.2481,26.0
2,2022-09-21,29.960,30.6575,25.5120,29.2175,257.0
3,2022-09-20,29.750,30.6805,25.9390,29.2308,96.0
4,2022-09-19,29.040,30.7775,24.8089,29.1271,158.0
...,...,...,...,...,...,...
643,2020-03-06,20.046,22.7016,23.6370,18.2868,3.0
644,2020-03-05,27.060,22.6695,24.0394,18.2448,5.0
645,2020-03-04,26.127,22.3514,24.0017,18.2013,30.0
646,2020-03-03,25.820,22.1294,24.1098,18.1132,NaN


# Compute Standard Deviations of Polarity and Subjectivity

In [36]:
articles_std_1 = combined_news_1.resample('1D', on='date')[['Polarity', 'Subjectivity', 'Content Polarity', 'Content Subjectivity']].std().reset_index()
articles_std_1.dropna(inplace=True)
articles_std_1

,date,Polarity,Subjectivity,Content Polarity,Content Subjectivity
1027,2012-01-28,0.207615,0.297288,0.324906,0.301097
1028,2012-01-29,0.317304,0.371966,0.139138,0.240321
1034,2012-02-04,0.300415,0.326006,0.166929,0.311393
1035,2012-02-05,0.213640,0.313416,0.265383,0.300798
1041,2012-02-11,0.257337,0.340435,0.177957,0.345630
...,...,...,...,...,...
4914,2022-09-19,0.105018,0.206479,0.000000,0.000000
4915,2022-09-20,0.000000,0.000000,0.057735,0.230940
4916,2022-09-21,0.020412,0.142887,0.268144,0.211766
4917,2022-09-22,0.377635,0.333437,0.182484,0.170453


In [37]:
articles_std_1.rename(columns={'date': 'Date', 'Polarity':'Headline Polarity', 'Subjectivity':'Headline Subjectivity'}, inplace=True)

# Merge Sentiment and Implied Volatility Data

In [38]:
merged1 = pd.merge(spyIV, articles_std_1, on='Date')

In [39]:
merged1

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,27.2500,23.3130,24.7004,21.9399,13803193.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,25.2500,22.9190,24.6545,21.9436,8510835.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,26.1400,22.6710,23.2973,21.8277,9723268.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,25.1700,22.4105,23.6350,21.8820,6950163.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,23.6600,22.1925,23.0622,21.8237,7426550.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1641,2012-03-06,16.5826,15.5209,NaN,NaN,NaN,0.229819,0.337685,0.274240,0.166796
1642,2012-03-05,14.3891,15.4710,NaN,NaN,NaN,0.364837,0.226202,0.260496,0.124036
1643,2012-03-02,13.7901,15.4917,NaN,NaN,NaN,0.376466,0.338847,0.165700,0.139576
1644,2012-03-01,13.9527,15.5841,NaN,NaN,NaN,0.242065,0.328613,0.214151,0.210019


In [76]:
merged1_covid = pd.merge(spyIV_COVID, articles_std_1, on = "Date")
merged1_covid

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,27.250,23.3130,24.7004,21.9399,13803193.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,25.250,22.9190,24.6545,21.9436,8510835.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,26.140,22.6710,23.2973,21.8277,9723268.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,25.170,22.4105,23.6350,21.8820,6950163.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,23.660,22.1925,23.0622,21.8237,7426550.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
151,2020-03-06,35.768,20.6823,22.5562,13.3505,7057538.0,0.031250,0.093750,0.216025,0.378686
152,2020-03-05,33.466,19.5214,21.1487,13.1126,5020112.0,0.131368,0.121218,0.047871,0.254406
153,2020-03-04,25.034,18.4997,19.9014,12.8533,5396965.0,0.171796,0.250000,0.096826,0.284001
154,2020-03-03,32.011,17.9503,18.4344,12.6761,5445981.0,0.360555,0.264575,0.081968,0.223723


In [40]:
merged2 = pd.merge(QQQ, articles_std_1, on = "Date")
merged2

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,33.3500,30.3280,29.7307,29.4133,3819144.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,31.5100,29.9660,29.6688,29.4100,2848974.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,32.2900,29.7390,28.8371,29.3421,3428578.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,31.6400,29.4970,29.3382,29.3909,1937901.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,30.1700,29.2825,29.0248,29.3496,2561835.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1643,2012-03-06,17.8164,16.8540,NaN,NaN,NaN,0.229819,0.337685,0.274240,0.166796
1644,2012-03-05,16.6065,16.8000,NaN,NaN,NaN,0.364837,0.226202,0.260496,0.124036
1645,2012-03-02,15.1673,16.7756,NaN,NaN,NaN,0.376466,0.338847,0.165700,0.139576
1646,2012-03-01,15.4686,16.8537,NaN,NaN,NaN,0.242065,0.328613,0.214151,0.210019


In [77]:
merged2_covid = pd.merge(QQQ_COVID, articles_std_1, on = "Date")
merged2_covid

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,33.350,30.3280,29.7307,29.4133,3819144.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,31.510,29.9660,29.6688,29.4100,2848974.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,32.290,29.7390,28.8371,29.3421,3428578.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,31.640,29.4970,29.3382,29.3909,1937901.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,30.170,29.2825,29.0248,29.3496,2561835.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
151,2020-03-06,39.998,24.8018,28.1611,17.2803,1169763.0,0.031250,0.093750,0.216025,0.378686
152,2020-03-05,37.492,23.6210,26.8479,17.0413,930637.0,0.131368,0.121218,0.047871,0.254406
153,2020-03-04,28.434,22.5849,25.6676,16.7968,1120002.0,0.171796,0.250000,0.096826,0.284001
154,2020-03-03,35.336,21.9879,24.0715,16.6002,1111158.0,0.360555,0.264575,0.081968,0.223723


In [54]:
merged3 = pd.merge(GSG, articles_std_1, on = "Date")
merged3

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,30.8900,30.6530,26.3471,29.2734,263.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,28.6700,30.6240,25.7979,29.2481,26.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,29.9600,30.6575,25.5120,29.2175,257.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,29.7500,30.6805,25.9390,29.2308,96.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,29.0400,30.7775,24.8089,29.1271,158.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1643,2012-03-06,22.3440,20.8231,NaN,NaN,NaN,0.229819,0.337685,0.274240,0.166796
1644,2012-03-05,20.8693,20.7784,NaN,NaN,NaN,0.364837,0.226202,0.260496,0.124036
1645,2012-03-02,19.8761,20.7767,NaN,NaN,NaN,0.376466,0.338847,0.165700,0.139576
1646,2012-03-01,19.1285,20.8532,NaN,NaN,NaN,0.242065,0.328613,0.214151,0.210019


In [78]:
merged3_covid = pd.merge(GSG_COVID, articles_std_1, on = "Date")
merged3_covid

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,30.890,30.6530,26.3471,29.2734,263.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,28.670,30.6240,25.7979,29.2481,26.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,29.960,30.6575,25.5120,29.2175,257.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,29.750,30.6805,25.9390,29.2308,96.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,29.040,30.7775,24.8089,29.1271,158.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
151,2020-03-06,20.046,22.7016,23.6370,18.2868,3.0,0.031250,0.093750,0.216025,0.378686
152,2020-03-05,27.060,22.6695,24.0394,18.2448,5.0,0.131368,0.121218,0.047871,0.254406
153,2020-03-04,26.127,22.3514,24.0017,18.2013,30.0,0.171796,0.250000,0.096826,0.284001
154,2020-03-03,25.820,22.1294,24.1098,18.1132,NaN,0.360555,0.264575,0.081968,0.223723


# Regression Analysis

We're now doing regression of Implied Volatility on the standard deviation of headline and content polarity and subjectivity.

## We first look at the IV data from SPY and all news sources

In [42]:
X = merged1[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged1[['IV30']]

In [52]:
import statsmodels.api as sm
X2 = sm.add_constant(X)
model = sm.OLS(y, X2)
model_fitted = model.fit()
model_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   IV30   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     6.410
Date:                Tue, 05 Dec 2023   Prob (F-statistic):           4.07e-05
Time:                        16:12:26   Log-Likelihood:                -4999.4
No. Observations:                1646   AIC:                         1.001e+04
Df Residuals:                    1641   BIC:                         1.004e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    13.2822      0.417     31.849      0.000      12.464      14.100
Headline Polarity        -1.1585      1.175     -0.986      0.324      -3.463       1.146
Headline Subjectivity    -2.1684      1.227     -1.768      0.077      -4.575       0.238
Content Polarity         -1.5372      1.418     -1.084      0.279      -4.319       1.244
Content Subjectivity      5.3890      1.330      4.052      0.000       2.780       7.997
==============================================================================
Omnibus:                      978.617   Durbin-Watson:                   0.191
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17000.471
Skew:                           2.438   Prob(JB):                         0.00
Kurtosis:                      17.970   Cond. No.                         14.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
model_no_intercept = sm.OLS(y, X)
model_no_intercept_fitted = model_no_intercept.fit()
model_no_intercept_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   IV30   R-squared (uncentered):                   0.798
Model:                            OLS   Adj. R-squared (uncentered):              0.798
Method:                 Least Squares   F-statistic:                              1622.
Date:                Tue, 05 Dec 2023   Prob (F-statistic):                        0.00
Time:                        16:14:06   Log-Likelihood:                         -5395.5
No. Observations:                1646   AIC:                                  1.080e+04
Df Residuals:                    1642   BIC:                                  1.082e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Headline Polarity         4.8422      1.475      3.283      0.001       1.949       7.735
Headline Subjectivity    17.1377      1.356     12.636      0.000      14.478      19.798
Content Polarity          9.1798      1.752      5.240      0.000       5.743      12.616
Content Subjectivity     23.6737      1.526     15.518      0.000      20.682      26.666
==============================================================================
Omnibus:                      449.674   Durbin-Watson:                   0.730
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2398.479
Skew:                           1.175   Prob(JB):                         0.00
Kurtosis:                       8.427   Cond. No.                         6.14
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
# We now look at COVID times data
X = merged1_covid[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged1_covid[['IV30']]

X2 = sm.add_constant(X)
model = sm.OLS(y, X2)
model_fitted = model.fit()
model_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   IV30   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.2617
Date:                Tue, 05 Dec 2023   Prob (F-statistic):              0.902
Time:                        17:08:15   Log-Likelihood:                -543.03
No. Observations:                 156   AIC:                             1096.
Df Residuals:                     151   BIC:                             1111.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    19.4988      1.733     11.252      0.000      16.075      22.923
Headline Polarity        -1.7807      5.520     -0.323      0.747     -12.687       9.126
Headline Subjectivity     1.7006      4.808      0.354      0.724      -7.800      11.201
Content Polarity          0.7890      6.284      0.126      0.900     -11.628      13.206
Content Subjectivity      4.6163      5.235      0.882      0.379      -5.727      14.959
==============================================================================
Omnibus:                      110.648   Durbin-Watson:                   0.320
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              927.466
Skew:                           2.536   Prob(JB):                    4.01e-202
Kurtosis:                      13.814   Cond. No.                         11.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
model_no_intercept = sm.OLS(y, X)
model_no_intercept_fitted = model_no_intercept.fit()
model_no_intercept_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   IV30   R-squared (uncentered):                   0.771
Model:                            OLS   Adj. R-squared (uncentered):              0.765
Method:                 Least Squares   F-statistic:                              127.9
Date:                Tue, 05 Dec 2023   Prob (F-statistic):                    1.36e-47
Time:                        17:11:00   Log-Likelihood:                         -590.52
No. Observations:                 156   AIC:                                      1189.
Df Residuals:                     152   BIC:                                      1201.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Headline Polarity         8.1465      7.364      1.106      0.270      -6.403      22.696
Headline Subjectivity    18.9477      6.159      3.076      0.002       6.779      31.116
Content Polarity         25.0706      7.977      3.143      0.002       9.311      40.830
Content Subjectivity     38.1146      5.819      6.550      0.000      26.618      49.611
==============================================================================
Omnibus:                       18.628   Durbin-Watson:                   1.139
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.493
Skew:                           0.640   Prob(JB):                     6.50e-07
Kurtosis:                       4.658   Cond. No.                         4.39
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Now, we look at the IV data from QQQ and all news sources

In [55]:
X = merged2[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged2[['IV30']]

In [56]:
X2 = sm.add_constant(X)
model = sm.OLS(y, X2)
model_fitted = model.fit()
model_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   IV30   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     18.91
Date:                Tue, 05 Dec 2023   Prob (F-statistic):           3.26e-15
Time:                        16:23:35   Log-Likelihood:                -5148.1
No. Observations:                1648   AIC:                         1.031e+04
Df Residuals:                    1643   BIC:                         1.033e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    16.3461      0.453     36.121      0.000      15.458      17.234
Headline Polarity        -2.5319      1.281     -1.977      0.048      -5.044      -0.020
Headline Subjectivity    -3.4470      1.336     -2.579      0.010      -6.068      -0.826
Content Polarity         -2.6882      1.546     -1.739      0.082      -5.720       0.344
Content Subjectivity     10.2261      1.449      7.058      0.000       7.384      13.068
==============================================================================
Omnibus:                      732.605   Durbin-Watson:                   0.205
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4447.602
Skew:                           2.001   Prob(JB):                         0.00
Kurtosis:                       9.982   Cond. No.                         14.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
model_no_intercept = sm.OLS(y, X)
model_no_intercept_fitted = model_no_intercept.fit()
model_no_intercept_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   IV30   R-squared (uncentered):                   0.824
Model:                            OLS   Adj. R-squared (uncentered):              0.823
Method:                 Least Squares   F-statistic:                              1918.
Date:                Tue, 05 Dec 2023   Prob (F-statistic):                        0.00
Time:                        16:24:05   Log-Likelihood:                         -5629.8
No. Observations:                1648   AIC:                                  1.127e+04
Df Residuals:                    1644   BIC:                                  1.129e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Headline Polarity         4.8214      1.693      2.848      0.004       1.500       8.142
Headline Subjectivity    20.3599      1.557     13.078      0.000      17.306      23.413
Content Polarity         10.4768      2.011      5.209      0.000       6.532      14.422
Content Subjectivity     32.7278      1.752     18.685      0.000      29.292      36.163
==============================================================================
Omnibus:                      305.132   Durbin-Watson:                   0.831
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              705.312
Skew:                           1.028   Prob(JB):                    6.97e-154
Kurtosis:                       5.458   Cond. No.                         6.14
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [83]:
X = merged2_covid[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged2_covid[['IV30']]

X2 = sm.add_constant(X)
model = sm.OLS(y, X2)
model_fitted = model.fit()
model_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   IV30   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.3834
Date:                Tue, 05 Dec 2023   Prob (F-statistic):              0.820
Time:                        17:15:19   Log-Likelihood:                -531.60
No. Observations:                 156   AIC:                             1073.
Df Residuals:                     151   BIC:                             1088.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    25.7141      1.611     15.966      0.000      22.532      28.896
Headline Polarity        -1.4098      5.130     -0.275      0.784     -11.546       8.727
Headline Subjectivity     0.5709      4.469      0.128      0.899      -8.258       9.400
Content Polarity          2.2114      5.841      0.379      0.705      -9.328      13.751
Content Subjectivity      4.9997      4.865      1.028      0.306      -4.613      14.612
==============================================================================
Omnibus:                       52.852   Durbin-Watson:                   0.310
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              169.110
Skew:                           1.294   Prob(JB):                     1.90e-37
Kurtosis:                       7.395   Cond. No.                         11.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
model_no_intercept = sm.OLS(y, X)
model_no_intercept_fitted = model_no_intercept.fit()
model_no_intercept_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   IV30   R-squared (uncentered):                   0.819
Model:                            OLS   Adj. R-squared (uncentered):              0.814
Method:                 Least Squares   F-statistic:                              171.8
Date:                Tue, 05 Dec 2023   Prob (F-statistic):                    2.52e-55
Time:                        17:15:23   Log-Likelihood:                         -608.74
No. Observations:                 156   AIC:                                      1225.
Df Residuals:                     152   BIC:                                      1238.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Headline Polarity        11.6817      8.276      1.411      0.160      -4.669      28.033
Headline Subjectivity    23.3156      6.922      3.368      0.001       9.641      36.991
Content Polarity         34.2329      8.964      3.819      0.000      16.522      51.943
Content Subjectivity     49.1758      6.540      7.520      0.000      36.255      62.096
==============================================================================
Omnibus:                        0.004   Durbin-Watson:                   1.442
Prob(Omnibus):                  0.998   Jarque-Bera (JB):                0.041
Skew:                          -0.007   Prob(JB):                        0.980
Kurtosis:                       2.922   Cond. No.                         4.39
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Now, we look at the IV data from GSG and all news sources

In [60]:
X = merged3[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged3[['IV30']]

In [59]:
X2 = sm.add_constant(X)
model = sm.OLS(y, X2)
model_fitted = model.fit()
model_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   IV30   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     3.288
Date:                Tue, 05 Dec 2023   Prob (F-statistic):             0.0108
Time:                        16:40:47   Log-Likelihood:                -6231.7
No. Observations:                1648   AIC:                         1.247e+04
Df Residuals:                    1643   BIC:                         1.250e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    24.0498      0.873     27.536      0.000      22.337      25.763
Headline Polarity        -1.4658      2.472     -0.593      0.553      -6.314       3.383
Headline Subjectivity     1.8927      2.579      0.734      0.463      -3.167       6.952
Content Polarity         -6.8339      2.983     -2.291      0.022     -12.685      -0.982
Content Subjectivity     -3.8358      2.796     -1.372      0.170      -9.320       1.649
==============================================================================
Omnibus:                     1416.204   Durbin-Watson:                   0.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            61939.934
Skew:                           3.793   Prob(JB):                         0.00
Kurtosis:                      32.060   Cond. No.                         14.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
model_no_intercept = sm.OLS(y, X)
model_no_intercept_fitted = model_no_intercept.fit()
model_no_intercept_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   IV30   R-squared (uncentered):                   0.731
Model:                            OLS   Adj. R-squared (uncentered):              0.730
Method:                 Least Squares   F-statistic:                              1118.
Date:                Tue, 05 Dec 2023   Prob (F-statistic):                        0.00
Time:                        16:42:46   Log-Likelihood:                         -6544.4
No. Observations:                1648   AIC:                                  1.310e+04
Df Residuals:                    1644   BIC:                                  1.312e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Headline Polarity         9.3529      2.949      3.171      0.002       3.568      15.138
Headline Subjectivity    36.9193      2.712     13.614      0.000      31.600      42.238
Content Polarity         12.5357      3.504      3.578      0.000       5.663      19.408
Content Subjectivity     29.2707      3.051      9.594      0.000      23.286      35.255
==============================================================================
Omnibus:                      940.778   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16514.700
Skew:                           2.298   Prob(JB):                         0.00
Kurtosis:                      17.811   Cond. No.                         6.14
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
X = merged3_covid[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged3_covid[['IV30']]

X2 = sm.add_constant(X)
model = sm.OLS(y, X2)
model_fitted = model.fit()
model_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   IV30   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.4664
Date:                Tue, 05 Dec 2023   Prob (F-statistic):              0.760
Time:                        17:18:44   Log-Likelihood:                -590.82
No. Observations:                 156   AIC:                             1192.
Df Residuals:                     151   BIC:                             1207.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    33.8683      2.354     14.387      0.000      29.217      38.520
Headline Polarity        -6.0434      7.499     -0.806      0.422     -20.860       8.773
Headline Subjectivity    -2.7846      6.532     -0.426      0.670     -15.691      10.121
Content Polarity         -2.3900      8.537     -0.280      0.780     -19.258      14.478
Content Subjectivity     -2.5172      7.112     -0.354      0.724     -16.568      11.534
==============================================================================
Omnibus:                       70.102   Durbin-Watson:                   0.136
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              214.237
Skew:                           1.821   Prob(JB):                     3.01e-47
Kurtosis:                       7.438   Cond. No.                         11.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
model_no_intercept = sm.OLS(y, X)
model_no_intercept_fitted = model_no_intercept.fit()
model_no_intercept_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   IV30   R-squared (uncentered):                   0.755
Model:                            OLS   Adj. R-squared (uncentered):              0.749
Method:                 Least Squares   F-statistic:                              117.1
Date:                Tue, 05 Dec 2023   Prob (F-statistic):                    2.16e-45
Time:                        17:19:16   Log-Likelihood:                         -658.15
No. Observations:                 156   AIC:                                      1324.
Df Residuals:                     152   BIC:                                      1337.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Headline Polarity        11.1996     11.361      0.986      0.326     -11.245      33.645
Headline Subjectivity    27.1727      9.501      2.860      0.005       8.401      45.944
Content Polarity         39.7859     12.305      3.233      0.002      15.475      64.097
Content Subjectivity     55.6677      8.977      6.201      0.000      37.932      73.403
==============================================================================
Omnibus:                       13.539   Durbin-Watson:                   1.016
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.646
Skew:                           0.656   Prob(JB):                     0.000660
Kurtosis:                       3.730   Cond. No.                         4.39
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Correlation Analysis

We now analyze the correlation between news sentiment and implied volatility from different companies.

In [90]:
import altair as alt

## We first look at the SPY IV data and all news sources

In [96]:
cormat2 = merged1[['IV30', 'Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']].corr()
cormat_tidy_2 = cormat2.stack().reset_index().rename(columns={0:'correlation'})
heatmap2 = alt.Chart(cormat_tidy_2).mark_rect().encode(
    x = alt.X('level_0').axis().title(''),
    y = alt.Y('level_1').axis().title(''),
    color = 'correlation'
).properties(
    width = 200,
    height = 200
)

heatmap2.display()

alt.Chart(...)

In [92]:
import pingouin as pg

pg.corr(merged1['IV30'], merged1['Headline Polarity'])

,n,r,CI95%,p-val,BF10,power
pearson,1646,-0.060373,"[-0.11, -0.01]",0.014294,0.619,0.688238


In [93]:
pg.corr(merged1['IV30'], merged1['Content Polarity'])

,n,r,CI95%,p-val,BF10,power
pearson,1646,0.017312,"[-0.03, 0.07]",0.482755,0.04,0.108071


In [94]:
pg.corr(merged1['IV30'], merged1['Headline Subjectivity'])

,n,r,CI95%,p-val,BF10,power
pearson,1646,-0.064255,"[-0.11, -0.02]",0.009117,0.922,0.741789


In [95]:
pg.corr(merged1['IV30'], merged1['Content Subjectivity'])

,n,r,CI95%,p-val,BF10,power
pearson,1646,0.098326,"[0.05, 0.15]",0.000064,89.893,0.979306


In [36]:
merged1.to_csv("SPYIV_and_sentiment.csv")

# QQQ Data

In [37]:
QQQ = pd.read_csv("HistoricalIV_QQQ.csv")
QQQ

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,12/1/2023,15.0200,15.7985,14.5423,19.7775,3210496.0
1,11/30/2023,14.9900,15.9505,15.1274,19.7775,3857053.0
2,11/29/2023,14.7900,16.1700,14.9468,19.7865,3699101.0
3,11/28/2023,14.3400,16.4815,15.1918,19.7887,3274355.0
4,11/27/2023,14.3900,16.8700,15.7020,19.8024,2875740.0
...,...,...,...,...,...,...
3175,3/29/2011,19.0443,19.3225,NaN,NaN,NaN
3176,3/28/2011,20.2398,19.3920,NaN,NaN,NaN
3177,3/25/2011,18.7910,19.1095,NaN,NaN,NaN
3178,3/24/2011,18.4833,19.2687,NaN,NaN,NaN


In [38]:
QQQ['Date'] = pd.to_datetime(QQQ['Date'])

QQQ = QQQ[(QQQ['Date'] >= oldest_date) & (QQQ['Date'] <= most_recent_date)].reset_index(drop=True)

QQQ

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,33.3500,30.3280,29.7307,29.4133,3819144.0
1,2022-09-22,31.5100,29.9660,29.6688,29.4100,2848974.0
2,2022-09-21,32.2900,29.7390,28.8371,29.3421,3428578.0
3,2022-09-20,31.6400,29.4970,29.3382,29.3909,1937901.0
4,2022-09-19,30.1700,29.2825,29.0248,29.3496,2561835.0
...,...,...,...,...,...,...
2876,2011-03-29,19.0443,19.3225,NaN,NaN,NaN
2877,2011-03-28,20.2398,19.3920,NaN,NaN,NaN
2878,2011-03-25,18.7910,19.1095,NaN,NaN,NaN
2879,2011-03-24,18.4833,19.2687,NaN,NaN,NaN


In [39]:
merged2 = pd.merge(QQQ, articles_std_1, on='Date')
merged2

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,33.3500,30.3280,29.7307,29.4133,3819144.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,31.5100,29.9660,29.6688,29.4100,2848974.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,32.2900,29.7390,28.8371,29.3421,3428578.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,31.6400,29.4970,29.3382,29.3909,1937901.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,30.1700,29.2825,29.0248,29.3496,2561835.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1643,2012-03-06,17.8164,16.8540,NaN,NaN,NaN,0.229819,0.337685,0.274240,0.166796
1644,2012-03-05,16.6065,16.8000,NaN,NaN,NaN,0.364837,0.226202,0.260496,0.124036
1645,2012-03-02,15.1673,16.7756,NaN,NaN,NaN,0.376466,0.338847,0.165700,0.139576
1646,2012-03-01,15.4686,16.8537,NaN,NaN,NaN,0.242065,0.328613,0.214151,0.210019


In [42]:
X1 = merged2[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y1 = merged2[['IV30 20-Day MA']]
y12 = merged2[['IV30']]

In [43]:
model3 = LinearRegression().fit(X1, y1)
model4 = LinearRegression().fit(X1, y12)

print(model3.coef_, model3.intercept_)
print(model4.coef_, model4.intercept_)

[[-2.44251468 -3.35601533 -1.57183111  8.90196036]] [16.21649673]
[[-2.531891   -3.44695062 -2.68821906 10.22610445]] [16.34608189]


In [44]:
y1_pred = model3.predict(X)
y12_pred = model4.predict(X)

mse_3 = mean_squared_error(y1, y1_pred)
r2_3 = r2_score(y1, y1_pred)

mse_4 = mean_squared_error(y12, y12_pred)
r2_4 = r2_score(y12, y12_pred)

print(mse_3, r2_3)
print(mse_4, r2_4)

20.406541018423763 0.05383006784221034
30.260530677047843 0.044008132192666194


In [69]:
cormat3 = merged2[['IV30 20-Day MA', 'Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']].corr()
cormat_tidy3 = cormat3.stack().reset_index().rename(columns={0:'corr'})
heatmap3 = alt.Chart(cormat_tidy3).mark_rect().encode(
    x = alt.X('level_0').axis().title(''),
    y = alt.Y('level_1').axis().title(''),
    color = 'corr'
).properties(
    width = 200,
    height = 200
)

heatmap3.display()

alt.Chart(...)

In [65]:
cormat4 = merged2[['IV30', 'Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']].corr()
cormat_tidy_4 = cormat4.stack().reset_index().rename(columns={0:'corr'})
heatmap4 = alt.Chart(cormat_tidy_4).mark_rect().encode(
    x = alt.X('level_0').axis().title(''),
    y = alt.Y('level_1').axis().title(''),
    color = 'corr'
).properties(
    width = 200,
    height = 200
)

heatmap4.display()

alt.Chart(...)

In [48]:
pg.corr(merged2['IV30'], merged2['Headline Polarity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,-0.103077,"[-0.15, -0.06]",0.000028,200.827,0.98735


In [50]:
pg.corr(merged2['IV30'], merged2['Headline Subjectivity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,-0.099513,"[-0.15, -0.05]",0.000052,110.246,0.981722


In [51]:
pg.corr(merged2['IV30'], merged2['Content Polarity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,0.033405,"[-0.01, 0.08]",0.175271,0.077,0.273294


In [52]:
pg.corr(merged2['IV30'], merged2['Content Subjectivity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,0.170519,"[0.12, 0.22]",3.211229e-12,1.056e+09,1.0


In [53]:
merged2.to_csv("QQQ_and_sentiment.csv")

# GSG

In [54]:
GSG = pd.read_csv("HistoricalIV_GSG.csv")
GSG

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,12/1/2023,19.8900,20.4440,21.3741,20.4524,9.0
1,11/30/2023,20.3000,20.4900,19.9467,20.3568,380.0
2,11/29/2023,20.2800,20.5290,19.9720,20.4898,7.0
3,11/28/2023,20.3800,20.6120,20.1599,20.5053,31.0
4,11/27/2023,21.5300,20.7540,20.3500,20.5487,328.0
...,...,...,...,...,...,...
4150,5/15/2007,21.6852,21.5287,NaN,NaN,NaN
4151,5/14/2007,21.6950,21.4896,NaN,NaN,NaN
4152,5/11/2007,20.9690,21.4211,NaN,NaN,NaN
4153,5/10/2007,21.4735,21.6472,NaN,NaN,NaN


In [55]:
GSG['Date'] = pd.to_datetime(GSG['Date'])

GSG = GSG[(GSG['Date'] >= oldest_date) & (GSG['Date'] <= most_recent_date)].reset_index(drop=True)

GSG

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,30.8900,30.6530,26.3471,29.2734,263.0
1,2022-09-22,28.6700,30.6240,25.7979,29.2481,26.0
2,2022-09-21,29.9600,30.6575,25.5120,29.2175,257.0
3,2022-09-20,29.7500,30.6805,25.9390,29.2308,96.0
4,2022-09-19,29.0400,30.7775,24.8089,29.1271,158.0
...,...,...,...,...,...,...
3370,2009-04-13,42.1140,43.2152,NaN,NaN,NaN
3371,2009-04-09,40.8940,43.3351,NaN,NaN,NaN
3372,2009-04-08,42.8479,43.4892,NaN,NaN,NaN
3373,2009-04-07,43.5973,43.6493,NaN,NaN,NaN


In [56]:
merged3 = pd.merge(GSG, articles_std_1, on='Date')
merged3

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,30.8900,30.6530,26.3471,29.2734,263.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,28.6700,30.6240,25.7979,29.2481,26.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,29.9600,30.6575,25.5120,29.2175,257.0,0.020412,0.142887,0.268144,0.211766
3,2022-09-20,29.7500,30.6805,25.9390,29.2308,96.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,29.0400,30.7775,24.8089,29.1271,158.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1643,2012-03-06,22.3440,20.8231,NaN,NaN,NaN,0.229819,0.337685,0.274240,0.166796
1644,2012-03-05,20.8693,20.7784,NaN,NaN,NaN,0.364837,0.226202,0.260496,0.124036
1645,2012-03-02,19.8761,20.7767,NaN,NaN,NaN,0.376466,0.338847,0.165700,0.139576
1646,2012-03-01,19.1285,20.8532,NaN,NaN,NaN,0.242065,0.328613,0.214151,0.210019


In [59]:
X2 = merged3[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y21 = merged3[['IV30 20-Day MA']]
y22 = merged3[['IV30']]

In [60]:
model5 = LinearRegression().fit(X2, y21)
model6 = LinearRegression().fit(X2, y22)

print(model5.coef_, model5.intercept_)
print(model6.coef_, model6.intercept_)

[[-2.63860219  4.18305133 -3.80552576 -6.38590714]] [23.51880998]
[[-1.4658492   1.89266287 -6.83388305 -3.83576033]] [24.04975492]


In [61]:
y21_pred = model5.predict(X)
y22_pred = model6.predict(X)

mse_5 = mean_squared_error(y21, y21_pred)
r2_5 = r2_score(y21, y21_pred)

mse_6 = mean_squared_error(y22, y22_pred)
r2_6 = r2_score(y22, y22_pred)

print(mse_5, r2_5)
print(mse_6, r2_6)

71.16993191066487 0.013662175988843783
112.71685328615038 0.007940504761197675


In [70]:
cormat5 = merged3[['IV30 20-Day MA', 'Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']].corr()
cormat_tidy5 = cormat5.stack().reset_index().rename(columns={0:'corr'})
heatmap5 = alt.Chart(cormat_tidy5).mark_rect().encode(
    x = alt.X('level_0').axis().title(''),
    y = alt.Y('level_1').axis().title(''),
    color = 'corr'
).properties(
    width = 200,
    height = 200
)

heatmap5.display()

alt.Chart(...)

In [71]:
cormat6 = merged3[['IV30', 'Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']].corr()
cormat_tidy_6 = cormat6.stack().reset_index().rename(columns={0:'corr'})
heatmap6 = alt.Chart(cormat_tidy_6).mark_rect().encode(
    x = alt.X('level_0').axis().title(''),
    y = alt.Y('level_1').axis().title(''),
    color = 'corr'
).properties(
    width = 200,
    height = 200
)

heatmap6.display()

alt.Chart(...)

In [72]:
pg.corr(merged3['IV30'], merged3['Headline Polarity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,-0.008951,"[-0.06, 0.04]",0.716524,0.033,0.06523


In [73]:
pg.corr(merged3['IV30'], merged3['Headline Subjectivity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,0.010863,"[-0.04, 0.06]",0.659445,0.034,0.072524


In [74]:
pg.corr(merged3['IV30'], merged3['Content Polarity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,-0.080791,"[-0.13, -0.03]",0.001028,6.719,0.907395


In [76]:
pg.corr(merged3['IV30'], merged3['Content Subjectivity'])

,n,r,CI95%,p-val,BF10,power
pearson,1648,-0.064981,"[-0.11, -0.02]",0.008322,1.0,0.751749


In [78]:
merged3.to_csv("GSG_and_sentiment.csv")

# We try using different combinations of news sources and options data

First, we try combining US news, world news, and business news together and we examine the relationship of standard deviations of polarity and subjectivity and implied volatility of SPY ticker. 

In [79]:
combined_news_us_world_business = pd.concat([US_news, business_news, World_news], axis=0)

In [81]:
combined_news_us_world_business.drop(columns=['Unnamed: 0', 'Heading', 'Article', 'short_description', 'headline', 'category'], inplace=True)
combined_news_us_world_business

,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,0.000000,0.000000,0.377273,0.118182,2022-09-23 00:00:00
1,0.200000,0.100000,0.000000,0.000000,2022-09-23 00:00:00
2,0.266667,-0.233333,0.550000,-0.233333,2022-09-22 00:00:00
3,0.400000,-0.200000,0.225000,-0.200000,2022-09-22 00:00:00
4,0.750000,0.318182,0.577273,-0.131818,2022-09-22 00:00:00
...,...,...,...,...,...
4341,0.100000,0.150000,0.398691,0.158471,2017-03-25 00:00:00
4342,0.000000,0.000000,0.302778,0.043182,2017-03-25 00:00:00
4343,0.000000,0.000000,0.173333,0.110000,2017-03-26 00:00:00
4344,0.454545,0.136364,0.408018,0.091746,2017-03-27 00:00:00


In [83]:
combined_news_us_world_business['date'] = pd.to_datetime(combined_news_us_world_business['date'])
combined_news_us_world_business

,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,0.000000,0.000000,0.377273,0.118182,2022-09-23
1,0.200000,0.100000,0.000000,0.000000,2022-09-23
2,0.266667,-0.233333,0.550000,-0.233333,2022-09-22
3,0.400000,-0.200000,0.225000,-0.200000,2022-09-22
4,0.750000,0.318182,0.577273,-0.131818,2022-09-22
...,...,...,...,...,...
4341,0.100000,0.150000,0.398691,0.158471,2017-03-25
4342,0.000000,0.000000,0.302778,0.043182,2017-03-25
4343,0.000000,0.000000,0.173333,0.110000,2017-03-26
4344,0.454545,0.136364,0.408018,0.091746,2017-03-27


In [84]:
oldest_date = combined_news_us_world_business['date'].min()
most_recent_date = combined_news_us_world_business['date'].max()

In [85]:
spyIV = spyIV[(spyIV['Date'] >= oldest_date) & (spyIV['Date'] <= most_recent_date)].reset_index(drop=True)

spyIV

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,27.2500,23.3130,24.7004,21.9399,13803193.0
1,2022-09-22,25.2500,22.9190,24.6545,21.9436,8510835.0
2,2022-09-21,26.1400,22.6710,23.2973,21.8277,9723268.0
3,2022-09-20,25.1700,22.4105,23.6350,21.8820,6950163.0
4,2022-09-19,23.6600,22.1925,23.0622,21.8237,7426550.0
...,...,...,...,...,...,...
2659,2012-02-03,14.8038,16.8302,NaN,NaN,NaN
2660,2012-02-02,15.6365,17.0157,NaN,NaN,NaN
2661,2012-02-01,16.1945,17.1966,NaN,NaN,NaN
2662,2012-01-31,17.0607,17.3980,NaN,NaN,NaN


In [96]:
articles_std_us_world_business = combined_news_us_world_business.resample('1D', on='date')[['Polarity', 'Subjectivity', 'Content Polarity', 'Content Subjectivity']].std().reset_index()
articles_std_us_world_business.dropna(inplace=True)
articles_std_us_world_business

,date,Polarity,Subjectivity,Content Polarity,Content Subjectivity
0,2012-01-28,0.209032,0.244865,0.278444,0.422973
1,2012-01-29,0.401930,0.407324,0.063918,0.274344
7,2012-02-04,0.241823,0.255270,0.167708,0.269898
8,2012-02-05,0.237327,0.333153,0.245344,0.243724
14,2012-02-11,0.320156,0.254440,0.207299,0.390478
...,...,...,...,...,...
3887,2022-09-19,0.105018,0.206479,0.000000,0.000000
3888,2022-09-20,0.000000,0.000000,0.057735,0.230940
3889,2022-09-21,0.022361,0.156525,0.250000,0.202622
3890,2022-09-22,0.377635,0.333437,0.182484,0.170453


In [97]:
articles_std_us_world_business.rename(columns={'date': 'Date', 'Polarity':'Headline Polarity', 'Subjectivity':'Headline Subjectivity'}, inplace=True)
articles_std_us_world_business

,Date,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2012-01-28,0.209032,0.244865,0.278444,0.422973
1,2012-01-29,0.401930,0.407324,0.063918,0.274344
7,2012-02-04,0.241823,0.255270,0.167708,0.269898
8,2012-02-05,0.237327,0.333153,0.245344,0.243724
14,2012-02-11,0.320156,0.254440,0.207299,0.390478
...,...,...,...,...,...
3887,2022-09-19,0.105018,0.206479,0.000000,0.000000
3888,2022-09-20,0.000000,0.000000,0.057735,0.230940
3889,2022-09-21,0.022361,0.156525,0.250000,0.202622
3890,2022-09-22,0.377635,0.333437,0.182484,0.170453


In [98]:
merged_us_world_business_spy = pd.merge(spyIV, articles_std_us_world_business, on='Date')
merged_us_world_business_spy

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,27.2500,23.3130,24.7004,21.9399,13803193.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,25.2500,22.9190,24.6545,21.9436,8510835.0,0.377635,0.333437,0.182484,0.170453
2,2022-09-21,26.1400,22.6710,23.2973,21.8277,9723268.0,0.022361,0.156525,0.250000,0.202622
3,2022-09-20,25.1700,22.4105,23.6350,21.8820,6950163.0,0.000000,0.000000,0.057735,0.230940
4,2022-09-19,23.6600,22.1925,23.0622,21.8237,7426550.0,0.105018,0.206479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1566,2012-03-06,16.5826,15.5209,NaN,NaN,NaN,0.229819,0.337685,0.274240,0.166796
1567,2012-03-05,14.3891,15.4710,NaN,NaN,NaN,0.364837,0.226202,0.260496,0.124036
1568,2012-03-02,13.7901,15.4917,NaN,NaN,NaN,0.376466,0.338847,0.165700,0.139576
1569,2012-03-01,13.9527,15.5841,NaN,NaN,NaN,0.242065,0.328613,0.214151,0.210019


In [102]:
X = merged_us_world_business_spy[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged_us_world_business_spy[['IV30']]

In [103]:
model = LinearRegression().fit(X, y)
print(model.coef_, model.intercept_)

[[ 0.15406679 -1.00189152 -1.45313974  1.66645654]] [13.51863839]


In [104]:
y_pred = model.predict(X)

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)
print(mse, r2)

26.20796195307951 0.001982219033464183


Now, we try only using tech and US news against the QQQ IV data.

In [105]:
combined_news_us_tech = pd.concat([US_news, tech_news], axis=0)

In [106]:
combined_news_us_tech.drop(columns=['Unnamed: 0', 'Heading', 'Article', 'short_description', 'headline', 'category'], inplace=True)
combined_news_us_tech

,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,0.000000,0.000000,0.377273,0.118182,2022-09-23
1,0.200000,0.100000,0.000000,0.000000,2022-09-23
2,0.266667,-0.233333,0.550000,-0.233333,2022-09-22
3,0.400000,-0.200000,0.225000,-0.200000,2022-09-22
4,0.750000,0.318182,0.577273,-0.131818,2022-09-22
...,...,...,...,...,...
2098,0.500000,0.000000,0.517857,0.354911,2012-01-28
2099,0.000000,0.000000,0.295833,0.041667,2012-01-28
2100,0.207407,0.031481,0.411111,0.316667,2012-01-28
2101,0.500000,0.500000,0.633333,0.366667,2012-01-28


In [107]:
combined_news_us_tech['date'] = pd.to_datetime(combined_news_us_tech['date'])
combined_news_us_tech

,Subjectivity,Polarity,Content Subjectivity,Content Polarity,date
0,0.000000,0.000000,0.377273,0.118182,2022-09-23
1,0.200000,0.100000,0.000000,0.000000,2022-09-23
2,0.266667,-0.233333,0.550000,-0.233333,2022-09-22
3,0.400000,-0.200000,0.225000,-0.200000,2022-09-22
4,0.750000,0.318182,0.577273,-0.131818,2022-09-22
...,...,...,...,...,...
2098,0.500000,0.000000,0.517857,0.354911,2012-01-28
2099,0.000000,0.000000,0.295833,0.041667,2012-01-28
2100,0.207407,0.031481,0.411111,0.316667,2012-01-28
2101,0.500000,0.500000,0.633333,0.366667,2012-01-28


In [108]:
oldest_date = combined_news_us_tech['date'].min()
most_recent_date = combined_news_us_tech['date'].max()

In [109]:
QQQ['Date'] = pd.to_datetime(QQQ['Date'])

QQQ = QQQ[(QQQ['Date'] >= oldest_date) & (QQQ['Date'] <= most_recent_date)].reset_index(drop=True)

QQQ

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume
0,2022-09-23,33.3500,30.3280,29.7307,29.4133,3819144.0
1,2022-09-22,31.5100,29.9660,29.6688,29.4100,2848974.0
2,2022-09-21,32.2900,29.7390,28.8371,29.3421,3428578.0
3,2022-09-20,31.6400,29.4970,29.3382,29.3909,1937901.0
4,2022-09-19,30.1700,29.2825,29.0248,29.3496,2561835.0
...,...,...,...,...,...,...
2661,2012-02-03,16.1194,17.7153,NaN,NaN,NaN
2662,2012-02-02,16.7281,17.8410,NaN,NaN,NaN
2663,2012-02-01,16.8481,17.9766,NaN,NaN,NaN
2664,2012-01-31,17.7274,18.1614,NaN,NaN,NaN


In [110]:
articles_std_us_tech = combined_news_us_tech.resample('1D', on='date')[['Polarity', 'Subjectivity', 'Content Polarity', 'Content Subjectivity']].std().reset_index()
articles_std_us_tech.dropna(inplace=True)
articles_std_us_tech

,date,Polarity,Subjectivity,Content Polarity,Content Subjectivity
0,2012-01-28,0.224050,0.335836,0.386047,0.185722
1,2012-01-29,0.000000,0.000000,0.210225,0.204869
7,2012-02-04,0.440959,0.416667,0.150041,0.413170
8,2012-02-05,0.058136,0.277261,0.336882,0.402854
14,2012-02-11,0.234521,0.400520,0.168758,0.335413
...,...,...,...,...,...
3884,2022-09-16,0.000000,0.000000,0.398730,0.375159
3887,2022-09-19,0.141421,0.282843,0.000000,0.000000
3889,2022-09-21,0.000000,0.000000,0.212132,0.318198
3890,2022-09-22,0.309244,0.249629,0.051745,0.195987


In [112]:
articles_std_us_tech.rename(columns={'date': 'Date', 'Polarity':'Headline Polarity', 'Subjectivity':'Headline Subjectivity'}, inplace=True)
articles_std_us_tech

,Date,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2012-01-28,0.224050,0.335836,0.386047,0.185722
1,2012-01-29,0.000000,0.000000,0.210225,0.204869
7,2012-02-04,0.440959,0.416667,0.150041,0.413170
8,2012-02-05,0.058136,0.277261,0.336882,0.402854
14,2012-02-11,0.234521,0.400520,0.168758,0.335413
...,...,...,...,...,...
3884,2022-09-16,0.000000,0.000000,0.398730,0.375159
3887,2022-09-19,0.141421,0.282843,0.000000,0.000000
3889,2022-09-21,0.000000,0.000000,0.212132,0.318198
3890,2022-09-22,0.309244,0.249629,0.051745,0.195987


In [113]:
merged_us_tech_qqq = pd.merge(QQQ, articles_std_us_tech, on='Date')
merged_us_tech_qqq

,Date,IV30,IV30 20-Day MA,OHLC 20-Day Vol,OHLC 52-Week Vol,Options Volume,Headline Polarity,Headline Subjectivity,Content Polarity,Content Subjectivity
0,2022-09-23,33.3500,30.3280,29.7307,29.4133,3819144.0,0.070711,0.141421,0.083567,0.266772
1,2022-09-22,31.5100,29.9660,29.6688,29.4100,2848974.0,0.309244,0.249629,0.051745,0.195987
2,2022-09-21,32.2900,29.7390,28.8371,29.3421,3428578.0,0.000000,0.000000,0.212132,0.318198
3,2022-09-19,30.1700,29.2825,29.0248,29.3496,2561835.0,0.141421,0.282843,0.000000,0.000000
4,2022-09-16,31.1100,29.0025,28.6062,29.3233,3569397.0,0.000000,0.000000,0.398730,0.375159
...,...,...,...,...,...,...,...,...,...,...
555,2015-01-06,18.4450,15.9820,16.1433,14.6895,669439.0,0.035355,0.141421,0.012276,0.040816
556,2013-12-24,12.0054,12.9813,10.2929,12.6644,99589.0,0.352252,0.245078,0.267777,0.318348
557,2013-11-29,11.8709,12.5919,12.0582,12.9209,271515.0,0.579811,0.169167,0.080263,0.180404
558,2012-12-24,18.4502,17.2396,NaN,NaN,NaN,0.077403,0.184046,0.223607,0.349870


In [120]:
X = merged_us_tech_qqq[['Headline Polarity', 'Headline Subjectivity', 'Content Polarity', 'Content Subjectivity']]
y = merged_us_tech_qqq[['IV30']]

In [118]:
model2 = LinearRegression().fit(X, y)
print(model2.coef_, model2.intercept_)

[[-3.36341903 -0.75293393 -3.22361254  4.2664654 ]] [19.285571]


In [119]:
y_pred = model2.predict(X)

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(mse, r2)

44.80890388448366 0.01597115422813178
